In [2]:
import numpy as np 
import math
from collections import defaultdict

In [45]:
M = 4096
N = 4096
K = 2048
BK = 128
BM = 128 
BN = 128
num_sms = 170
GM = M//BM
GN = N//BN
GK = K//BK 
num_tiles = GM*GN 

sm_counts = [0]*170
n_tiles_hit = 0
tile_set = set()
sm_map = defaultdict(list)



def tile_for(sm, step, num_sms, num_tiles): 
    val = (sm + step * num_sms)
    if val >= num_tiles: 
      return None
    else:
      return val

def tile_id_C_tile_coord(tile_id, GM,GN):
  assert math.log2(GM*GN) <= 32
  m = compact1by1(tile_id)
  n = compact1by1(tile_id >> 1)
  return (m,n)

def compact1by1(x):
    x &= 0x55555555                # keep even bits
    x = (x | (x >> 1)) & 0x33333333
    x = (x | (x >> 2)) & 0x0F0F0F0F
    x = (x | (x >> 4)) & 0x00FF00FF
    x = (x | (x >> 8)) & 0x0000FFFF
    return x


for step in range(math.ceil(num_tiles/num_sms)):
  for s in range(num_sms): 
    tile_id = tile_for(s,step,num_sms,num_tiles)
    if tile_id != None:
      if tile_id not in tile_set:
        tile_set.add(tile_id)
        sm_counts[s] += 1
        sm_map[s].append(tile_id)
      else:
        print("oopsie, two different sms or two different iterations hitting the same tile")
        break
    
  
  
def make_coords(sm_map,GM,GN):
  x = np.zeros((GM,GN)).astype(int)
  for sm_id in sm_map: 
    for tile_id in sm_map[sm_id]: 
      m,n = tile_id_C_tile_coord(tile_id, GM,GN)
      x[m,n] = sm_id
  return x
  

In [48]:
viz = make_coords(sm_map,GM,GN)
np.set_printoptions(
    threshold=np.inf,   # show all entries
    linewidth=200,      # wide lines
    edgeitems=10,       # keep edges if truncation happens
    suppress=True       # no scientific notation
)

In [49]:
print(viz)

[[  0   2   8  10  32  34  40  42 128 130 136 138 160 162 168   0   2   4  10  12  34  36  42  44 130 132 138 140 162 164   0   2]
 [  1   3   9  11  33  35  41  43 129 131 137 139 161 163 169   1   3   5  11  13  35  37  43  45 131 133 139 141 163 165   1   3]
 [  4   6  12  14  36  38  44  46 132 134 140 142 164 166   2   4   6   8  14  16  38  40  46  48 134 136 142 144 166 168   4   6]
 [  5   7  13  15  37  39  45  47 133 135 141 143 165 167   3   5   7   9  15  17  39  41  47  49 135 137 143 145 167 169   5   7]
 [ 16  18  24  26  48  50  56  58 144 146 152 154   6   8  14  16  18  20  26  28  50  52  58  60 146 148 154 156   8  10  16  18]
 [ 17  19  25  27  49  51  57  59 145 147 153 155   7   9  15  17  19  21  27  29  51  53  59  61 147 149 155 157   9  11  17  19]
 [ 20  22  28  30  52  54  60  62 148 150 156 158  10  12  18  20  22  24  30  32  54  56  62  64 150 152 158 160  12  14  20  22]
 [ 21  23  29  31  53  55  61  63 149 151 157 159  11  13  19  21  23  25  31  33  